In [37]:
#导入处理数据包
import numpy as np
import pandas as pd
#导入原始数据集
train=pd.read_csv("D:/train.csv")
#导入预测数据
test=pd.read_csv("D:/test.csv")
print('原始数据集：',train.shape, '预测数据集',test.shape)

原始数据集： (891, 12) 预测数据集 (418, 11)


In [38]:
full = train.append(test, ignore_index = True)
print('合并后的数据集：', full.shape)

合并后的数据集： (1309, 12)


In [39]:
full.describe()

,Age,Fare,Parch,PassengerId,Pclass,SibSp,Survived
count,1046.000000,1308.000000,1309.000000,1309.000000,1309.000000,1309.000000,891.000000
mean,29.881138,33.295479,0.385027,655.000000,2.294882,0.498854,0.383838
std,14.413493,51.758668,0.865560,378.020061,0.837836,1.041658,0.486592
min,0.170000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,21.000000,7.895800,0.000000,328.000000,2.000000,0.000000,0.000000
50%,28.000000,14.454200,0.000000,655.000000,3.000000,0.000000,0.000000
75%,39.000000,31.275000,0.000000,982.000000,3.000000,1.000000,1.000000
max,80.000000,512.329200,9.000000,1309.000000,3.000000,8.000000,1.000000


In [40]:
#查看每一列的数据类型和数据总数
print('处理前：')
full.info()
#年龄（age）
full['Age'] = full['Age'].fillna(full['Age'].mean())
#船票的价格（Fare）
full['Fare']=full['Fare'].fillna(full['Fare'].mean())
print('处理后')
full.info()

处理前：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 122.8+ KB
处理后
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 12 columns):
Age            1309 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1309 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int6

In [41]:
#登船港口（embarked）:查看里面数据
print(full['Embarked'].unique())
print(full['Embarked'].describe())

['S' 'C' 'Q' nan]
count     1307
unique       3
top          S
freq       914
Name: Embarked, dtype: object


In [42]:
print(full['Embarked'].head())

0    S
1    C
2    S
3    S
4    S
Name: Embarked, dtype: object


In [43]:
full['Embarked']=full['Embarked'].fillna('S')
full['Embarked'].shape[0]

1309

In [44]:
full['Cabin'].head()

0     NaN
1     C85
2     NaN
3    C123
4     NaN
Name: Cabin, dtype: object

In [45]:
full['Cabin']=full['Cabin'].fillna('U')
full['Cabin'].shape[0]

1309

In [46]:
#查看性别数据这一列
full['Sex'].head()

0      male
1    female
2    female
3    female
4      male
Name: Sex, dtype: object

In [47]:
'''
将性别的值映射为数值
男（male）对应数值1，女（female）对应数值0
'''
sex_mapDict = {'male': 1, 'female': 0}
#map函数：对Series每个数据应用自定义的函数计算
full['Sex'] = full['Sex'].map(sex_mapDict)
full['Sex'].head()

0    1
1    0
2    0
3    0
4    1
Name: Sex, dtype: int64

In [48]:
#查看该类数据内容
full['Embarked'].head()

0    S
1    C
2    S
3    S
4    S
Name: Embarked, dtype: object

In [49]:
embarkedDf = pd.DataFrame()
'''
使用get_dummies进行one-hot编码，产生虚拟变量（dummy variables）列名前缀prefix是Embarked
'''
embarkedDf = pd.get_dummies(full['Embarked'], prefix = 'Embarked')
embarkedDf.head()

,Embarked_C,Embarked_Q,Embarked_S
0,0,0,1
1,1,0,0
2,0,0,1
3,0,0,1
4,0,0,1


In [50]:
#添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克数据集full中
full = pd.concat([full, embarkedDf], axis = 1)
full.drop('Embarked', axis = 1, inplace = True)

In [51]:
full['Name'].head()

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
Name: Name, dtype: object

In [52]:
'''
定义函数：从姓名中获取头衔
'''
def getTitle(name):
    str1 = name.split(',')[1]  #Mr. Owen Harris
    str2 = str1.split('.')[0]  #Mr
    str3 = str2.strip()
    return str3

In [61]:
#存放提取后的特征
titleDf = pd.DataFrame()
#map函数：对Series每个数据应用自定义的函数计算
titleDf['Title'] = full['Name'].map(getTitle)
#查看数据详情
titleDf.head()

,Title
0,Mr
1,Mrs
2,Miss
3,Mrs
4,Mr


In [62]:
print(titleDf['Title'].describe())
print(titleDf['Title'].unique())

count     1309
unique      18
top         Mr
freq       757
Name: Title, dtype: object
['Mr' 'Mrs' 'Miss' 'Master' 'Don' 'Rev' 'Dr' 'Mme' 'Ms' 'Major' 'Lady'
 'Sir' 'Mlle' 'Col' 'Capt' 'the Countess' 'Jonkheer' 'Dona']


In [63]:
#姓名中头衔字符串与定义头衔类别的映射关系
title_mapDict = {
                    "Capt":        "Officer",
                    "Col":         "Officer",
                    "Major":       "Officer",
                    "Jonkheer":    "Royalty",
                    "Don":         "Royalty",
                    "Sir":         "Royalty",
                    "Dr":          "Officer",
                    "Rev":         "Officer",
                    "the Countess":"Royalty",
                    "Dona":        "Royalty",
                    "Mme":         "Mrs",
                    "Mlle":        "Miss",
                    "Ms":          "Mrs",
                    "Mr":          "Mr",
                    "Mrs":         "Mrs",
                    "Miss":        "Miss",
                    "Master":      "Master",
                    "Lady":        "Royalty"
                }
#map函数：对Series每个数据应用自定义的函数计算
titleDf['Title'] = titleDf['Title'].map(title_mapDict)

In [64]:
titleDf = pd.get_dummies(titleDf['Title'], prefix = 'Title')
titleDf.head()

,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Officer,Title_Royalty
0,0,0,1,0,0,0
1,0,0,0,1,0,0
2,0,1,0,0,0,0
3,0,0,0,1,0,0
4,0,0,1,0,0,0


In [65]:
#添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克数据集full中
full = pd.concat([full, titleDf], axis = 1)
full.drop('Name', axis = 1, inplace = True)

In [69]:
'''
客舱号的首字母是客舱的类别
'''
#查看客舱号的内容‘
full['Cabin'].head()

0       U
1     C85
2       U
3    C123
4       U
Name: Cabin, dtype: object

In [70]:
full['Cabin'] = full['Cabin'].map(lambda c:c[0])

In [71]:
print(full['Cabin'].describe())
print(full['Cabin'].unique())

count     1309
unique       9
top          U
freq      1014
Name: Cabin, dtype: object
['U' 'C' 'E' 'G' 'D' 'A' 'B' 'F' 'T']


In [72]:
cabinDf = pd.DataFrame()
'''
使用get_dummies进行one-hot编码，产生虚拟变量（dummy variables）列名前缀prefix是Embarked
'''
cabinDf = pd.get_dummies(full['Cabin'], prefix = 'Cabin')
cabinDf.head()

,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Cabin_U
0,0,0,0,0,0,0,0,0,1
1,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1
3,0,0,1,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1


In [75]:
#添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克数据集full中
full = pd.concat([full, cabinDf], axis = 1)
full.drop('Cabin', axis = 1, inplace = True)

In [74]:
#存放家庭信息
familyDf = pd.DataFrame()
#家庭人数
familyDf['FamilySize'] = full['Parch'] + full['SibSp'] + 1

#if条件为真的时候返回if前面的内容，否则返回0
familyDf['Family_Single'] = familyDf['FamilySize'].map(lambda s:1 if s==1 else 0)
familyDf['Family_Small'] = familyDf['FamilySize'].map(lambda s:1 if 2 <= s <= 4 else 0)
familyDf['Family_Large'] = familyDf['FamilySize'].map(lambda s:1 if 5 <= s else 0)
familyDf.head()

,FamilySize,Family_Single,Family_Small,Family_Large
0,2,0,1,0
1,2,0,1,0
2,1,1,0,0
3,2,0,1,0
4,1,1,0,0


In [76]:
#添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克数据集full中
full = pd.concat([full, familyDf], axis = 1)
full.drop('Parch', axis = 1, inplace = True)
full.drop('SibSp', axis = 1, inplace = True)

In [82]:
pClassDf = pd.DataFrame()
pClassDf = pd.get_dummies(full['Pclass'], prefix = 'Pclass')
#添加one-hot编码产生的虚拟变量（dummy variables）到泰坦尼克数据集full中
full = pd.concat([full, pClassDf], axis = 1)
full.drop('Pclass', axis = 1, inplace = True)

In [83]:
'''
查看各个特征与生成情况（Survived)的相关系数，
ascending=False表示按降序排列
'''

#相关性矩阵
corrDf = full.corr()
corrDf['Survived'].sort_values(ascending = False)

Survived         1.000000
Title_Mrs        0.344935
Title_Miss       0.332795
Pclass_1         0.285904
Family_Small     0.279855
Fare             0.257307
Cabin_B          0.175095
Embarked_C       0.168240
Cabin_D          0.150716
Cabin_E          0.145321
Cabin_C          0.114652
Pclass_2         0.093349
Title_Master     0.085221
Cabin_F          0.057935
Title_Royalty    0.033391
Cabin_A          0.022287
FamilySize       0.016639
Cabin_G          0.016040
Embarked_Q       0.003650
PassengerId     -0.005007
Cabin_T         -0.026456
Title_Officer   -0.031316
Age             -0.070323
Family_Large    -0.125147
Embarked_S      -0.149683
Family_Single   -0.203367
Cabin_U         -0.316912
Pclass_3        -0.322308
Sex             -0.543351
Title_Mr        -0.549199
Name: Survived, dtype: float64

In [87]:
#特征选择
full_X = pd.concat([titleDf, #头衔
                    pClassDf, #客舱等级
                    familyDf, #家庭大小
                    full['Fare'], #船票价格
                    cabinDf, #船舱号
                    embarkedDf, #登船港口
                    full['Sex'] #性别
                   ],axis = 1)

In [88]:
#原始数据集有891行
sourceRow = 891

#原始数据集：特征
source_X = full_X.loc[0: sourceRow - 1, :]  #行号从0开始，所以减1
#原始数据集：标签
source_y = full.loc[0: sourceRow - 1, 'Survived']
#预测数据集：特征
pred_X = full_X.loc[sourceRow:,:]

In [92]:
'''
确保这里原始数据集取得是前891行的数据，不然后面模型会有错误
'''
#原始数据集有多少行
print('原始数据集行数：', source_X.shape[0])
#预测数据集有多少行
print('原始预测数据集行数：', pred_X.shape[0])

原始数据集行数： 891
原始预测数据集行数： 418


In [94]:
'''
从原始数据集（source）中拆分出训练数据集（用于模型训练train）和测试数据集（用于模型评估）
'''
from sklearn.cross_validation import train_test_split
train_X, test_X, train_y, test_y = train_test_split(source_X, source_y, train_size = .8)

#输出数据集大小
print('原始数据集特征：', source_X.shape,
      '训练数据集特征：', train_X.shape,
      '测试数据集特征：', test_X.shape)
print('原始数据集标签：', source_y.shape,
      '训练数据集标签：', train_y.shape,
      '测试数据集标签：', test_y.shape)

原始数据集特征： (891, 27) 训练数据集特征： (712, 27) 测试数据集特征： (179, 27)
原始数据集标签： (891,) 训练数据集标签： (712,) 测试数据集标签： (179,)


In [96]:
#第一步：导入算法
from sklearn.linear_model import LogisticRegression
#第二步：创建模型：逻辑回归（logistic regression）
model = LogisticRegression()
#第三步：训练模型
model.fit(train_X, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [98]:
#分类问题：score得到的是模型的正确率
model.score(test_X, test_y)

0.770949720670391

In [105]:
#对预测数据集中的生存情况进行预测
pred_Y = model.predict(pred_X)
pred_Y = pred_Y.astype(int)

#乘客id
passenger_id = full.loc[sourceRow:,'PassengerId']
#数据框：乘客id,预测生存情况的值
predDf = pd.DataFrame(
    {'PassengerId':passenger_id, 'Survived':pred_Y})
predDf.shape

(418, 2)

In [106]:
predDf.head()

,PassengerId,Survived
891,892,0
892,893,1
893,894,0
894,895,0
895,896,1


In [107]:
predDf.to_csv('titanic_pred.csv',index = False)